## 환경 설정 및 Langfuse 연결

### 필수 패키지 확인

**참고**: LangFuse 3.9.0 버전에서는 `CallbackHandler`가 `langfuse.langchain` 모듈에 위치합니다.
이전 버전과의 호환성을 위해 import 경로에 fallback 처리가 포함되어 있습니다.

### 참고문서
- Langfuse 가이드: https://langfuse.com/docs
- RAGAS 프레임워크: https://docs.ragas.io/

### 환경 변수 확인

Langfuse를 사용하기 위해 다음 환경 변수가 필요합니다:

- `LANGFUSE_PUBLIC_KEY`: Langfuse Public Key (pk-lf-로 시작)
- `LANGFUSE_SECRET_KEY`: Langfuse Secret Key (sk-lf-로 시작)
- `LANGFUSE_HOST`: Langfuse 서버 URL (기본값: http://localhost:3000, Cloud 기본값: https://us.cloud.langfuse.com)

In [45]:
import os

from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, filter_messages
from langchain_core.runnables import RunnableConfig
from langchain_tavily import TavilySearch
from langfuse.langchain import CallbackHandler
from openrouter_llm import create_embedding_model, create_openrouter_llm

load_dotenv()
llm = model = create_openrouter_llm("openai/gpt-4.1", temperature=0)
embeddings = create_embedding_model("openai/text-embedding-3-small")

### Langfuse 연결 테스트

In [6]:
# Langfuse 콜백 핸들러 초기화
# 참고: CallbackHandler는 public_key만 받고, secret_key와 host는 환경 변수에서 자동으로 읽습니다.
# 환경 변수가 설정되어 있다면 public_key만 전달해도 됩니다.
langfuse_handler = CallbackHandler(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    update_trace=True,
)
langfuse_handler

No Langfuse client with public key pk-lf-a62c70c5-57cc-4147-8d63-525204fa10f4 has been initialized. Skipping tracing for decorated function.


## Langfuse 콜백 통합

### 트레이스 이름 및 메타데이터 설정

**참고**: LangFuse 3.9.0에서는 `CallbackHandler`가 자동으로 트레이스를 생성합니다.
트레이스 이름과 메타데이터는 LangGraph 실행 시 `config`의 `metadata`를 통해 전달할 수 있습니다.

In [46]:
question = "RAGAS와 DeepEval의 차이점은 무엇인가요?"

# RunnableConfig에 콜백 추가
# 참고: CallbackHandler를 config에 전달하면 자동으로 LLM 호출, Tool 사용이 추적됩니다.
config = RunnableConfig(
    callbacks=[langfuse_handler],
    run_name="agent-execution",  # LangFuse에서 표시될 실행 이름
    metadata={
        "session": "day5-session3",
        "user": "sds-superman",
        "environment": "jupyter",
        "trace_name": "sds-day5-class-20251114-1",
    },
)

system_prompt = """주어진 도구를 무조건 활용해서 사용자가 입력한 질문에 대해 최대한 전문적이고 친절하게 답변해주세요."""

# TavilySearch 도구 정의
tavily_search = TavilySearch(max_result=5, topic="general", search_depth="advanced")

# 에이전트 생성
agent = create_agent(
    model=llm,
    tools=[tavily_search],
    system_prompt=system_prompt,
    name="langfuse-sample-agent",
)

# 그래프 실행
result_with_langfuse = agent.invoke(
    input={"messages": [HumanMessage(content=question)]},
    config=config,
)

import json

# 도구 사용 컨텍스트 추출
# ToolMessage.content는 JSON 문자열이므로 파싱 필요
content_str = filter_messages(result_with_langfuse["messages"], include_types=[ToolMessage])[
    0
].content

# JSON 파싱
content_dict = json.loads(content_str)

# results 배열에서 각 항목을 문자열로 변환
results = content_dict.get("results", [])
context = [
    f"Title: {r.get('title', 'N/A')}\nContent: {r.get('content', 'N/A')}\nURL: {r.get('url', 'N/A')}"
    for r in results
]

# 결과 추출
answer = result_with_langfuse["messages"][-1].content

print("=== Answer ===")
print(answer)
print("\n" + "=" * 100)
print("=== Search Contexts===")
for i, ctx in enumerate(context, 1):
    print(f"\n[Context {i}]")
    print(ctx)
    print("-" * 50)

=== Answer ===
RAGAS와 DeepEval은 둘 다 LLM(대형 언어 모델)·RAG(Retrieval-Augmented Generation) 시스템의 평가를 위한 오픈소스 라이브러리이지만, 주요 차이점은 다음과 같습니다.

---

### 1. 평가 목적과 기능의 범위

- **RAGAS**
  - 주로 RAG 시스템, 즉 '검색-증강 생성' 파이프라인의 품질 평가에 특화되어 있습니다.
  - 대표적으로 네 가지 평가 지표(RAGASAnswerRelevancy, RAGASFaithfulness, RAGASContextualPrecision, RAGASContextualRecall)로 정량적 수치를 제공합니다.
  - 데이터 기반의 실험적 평가, 혹은 빠른 실험에 적합하며, LangChain, LlamaIndex 등 RAG 프레임워크와 밀접하게 연동됩니다.
  - 커스터마이즈나 CI/CD, 엔터프라이즈 통합보다는 연구적, 데이터 분석적 실험에 적합합니다.
  - 숫자 중심의 메트릭 제공에 중점을 두지만, 결과 해석이 직관적이지 않을 수 있습니다.

- **DeepEval**
  - RAG뿐 아니라 챗봇, LLM 에이전트 등 다양한 LLM 어플리케이션 평가에 쓸 수 있습니다.
  - 기존 RAGAS의 평가 지표들도 포함하지만, pass/fail 기준 도입, 개발자 친화적인 테스트 통합(예, Pytest 연동), 다양한 맞춤형 평가 지표(예, 독성·데이터 유출 탐지 등 안전성 테스트) 기능을 제공합니다.
  - 커스텀 메트릭과 데이터셋 생성, CI/CD(지속적 통합/배포) 파이프라인에서의 자동화된 테스트, 기업용 협업·리포팅 지원 등 대규모 개발 환경에 최적화되어 있습니다.
  - 테스트 결과를 단순 수치(스코어) 외에, 합격/불합격 기준 등으로 직관적으로 해석하도록 돕습니다.

---

### 2. 사용성 및 커스터마이징

- RAGAS: 주로 연구자, 데이터과학자에게 어울리며, 개발경험이나 맞춤형 테스트, 확장성은 제한적입니다.
- DeepEval:

### RAGAS 평가

In [47]:
def evaluate_with_ragas(
    question: str, answer: str, contexts: list[str], ground_truth: str = None
) -> dict:
    """RAGAS 평가

    Args:
        question: 사용자 질문
        answer: LLM 생성 답변
        contexts: 검색된 컨텍스트 리스트
        ground_truth: 정답 (선택적, context_precision/context_recall 계산에 필요)

    Returns:
        RAGAS 평가 점수 딕셔너리
    """
    try:
        # RAGAS import
        from ragas import EvaluationDataset, SingleTurnSample, evaluate
        from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness

        # SingleTurnSample 생성
        sample_kwargs = {
            "user_input": question,
            "response": answer,
            "retrieved_contexts": contexts,
        }

        # ground_truth가 있으면 reference 추가
        if ground_truth:
            sample_kwargs["reference"] = ground_truth

        sample = SingleTurnSample(**sample_kwargs)

        # EvaluationDataset 생성
        dataset = EvaluationDataset(samples=[sample])

        # 메트릭 선택 (ground_truth 유무에 따라)
        if ground_truth:
            metrics = [faithfulness, answer_relevancy, context_precision, context_recall]
        else:
            # ground_truth가 없으면 faithfulness와 answer_relevancy만 사용
            metrics = [faithfulness, answer_relevancy]

        # RAGAS 평가 실행
        result = evaluate(
            llm=llm,
            embeddings=embeddings,
            dataset=dataset,
            metrics=metrics,
        )

        # 결과 추출
        scores_df = result.to_pandas()
        scores = scores_df.iloc[0]

        faithfulness_score = float(scores.get("faithfulness", 0.0))
        relevancy_score = float(scores.get("answer_relevancy", 0.0))

        result_dict = {
            "faithfulness": faithfulness_score,
            "answer_relevancy": relevancy_score,
        }

        # ground_truth가 있으면 추가 메트릭 포함
        if ground_truth:
            precision_score = float(scores.get("context_precision", 0.0))
            recall_score = float(scores.get("context_recall", 0.0))
            result_dict["context_precision"] = precision_score
            result_dict["context_recall"] = recall_score
            passed = (
                faithfulness_score >= 0.7
                and relevancy_score >= 0.7
                and precision_score >= 0.7
                and recall_score >= 0.7
            )
        else:
            # ground_truth 없이는 faithfulness와 relevancy만 체크
            passed = faithfulness_score >= 0.7 and relevancy_score >= 0.7

        result_dict["passed"] = passed

        print("=== RAGAS 평가 결과 ===")
        print(f"- Faithfulness: {faithfulness_score:.3f}")
        print(f"- Answer Relevancy: {relevancy_score:.3f}")
        if ground_truth:
            print(f"- Context Precision: {precision_score:.3f}")
            print(f"- Context Recall: {recall_score:.3f}")
        print(f"- Passed: {passed}")

        return result_dict

    except Exception as e:
        print(f"RAGAS 평가 실패: {e}")
        import traceback

        traceback.print_exc()
        return {
            "faithfulness": 0.0,
            "answer_relevancy": 0.0,
            "passed": False,
            "error": str(e),
        }


# Ground truth 없이 평가
ragas_scores = evaluate_with_ragas(question, answer, context)

print("\n=== 평가 점수 요약 ===")
for metric, value in ragas_scores.items():
    if metric != "error":
        print(f"  {metric}: {value}")

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

=== RAGAS 평가 결과 ===
- Faithfulness: 0.938
- Answer Relevancy: 0.749
- Passed: True

=== 평가 점수 요약 ===
  faithfulness: 0.9375
  answer_relevancy: 0.7486281306894892
  passed: True


### Langfuse에 점수 기록

LangFuse `CallbackHandler`에 직접 `score()` 할 수 있는 메서드가 없습니다.
대신 별도의 `Langfuse` 클라이언트를 생성하고, `CallbackHandler.last_trace_id`를 사용하여
트레이스 ID를 가져온 후 `langfuse_client.score()`로 점수를 기록합니다.

In [48]:
from langfuse import Langfuse

langfuse_client = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST", "http://localhost:3000"),
)

# CallbackHandler에서 생성된 트레이스 ID 가져오기
# 참고: last_trace_id는 그래프 실행 후 자동으로 설정됩니다.
trace_id = langfuse_handler.last_trace_id

if trace_id:
    print(f"트레이스 ID: {trace_id}\n")

    # 각 스코어를 트레이스에 추가
    for metric, value in ragas_scores.items():
        if metric == "error":
            continue  # 에러는 스킵

        # 숫자 값 처리
        if isinstance(value, (int, float)):
            langfuse_client.create_score(
                trace_id=trace_id,
                name=metric,
                value=float(value),
                data_type="NUMERIC",  # data_type 필수
                comment=f"{metric} 평가 점수",
            )
            print(f"  ✓ {metric}: {value:.3f} (NUMERIC)")

        # Boolean 값 처리
        elif isinstance(value, bool):
            # Boolean은 0 또는 1로 변환
            langfuse_client.create_score(
                trace_id=trace_id,
                name=metric,
                value=1.0 if value else 0.0,
                data_type="BOOLEAN",  # data_type 명시
                comment=f"{metric} 평가 점수",
            )
            print(f"  ✓ {metric}: {value} → {1.0 if value else 0.0} (BOOLEAN)")

    # 데이터 flush (서버에 전송)
    langfuse_client.flush()

트레이스 ID: 00000000000000000000000000000000

  ✓ faithfulness: 0.938 (NUMERIC)
  ✓ answer_relevancy: 0.749 (NUMERIC)
  ✓ passed: 1.000 (NUMERIC)
